In [ ]:
# https://www.youtube.com/watch?v=rIV1EseKwU4&ab_channel=AIAnytime

In [1]:
!pip install -q chromadb
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers
!pip install -q langchain
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q bitsandbytes
!pip install -q trl
!pip install -q sentence_transformers
!pip install -q pypdf

In [2]:
!transformers-cli env


Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.39.2
- Platform: Windows-10-10.0.22621-SP0
- Python version: 3.11.8
- Huggingface_hub version: 0.22.1
- Safetensors version: 0.4.2
- Accelerate version: 0.28.0
- Accelerate config: 	not found
- PyTorch version (GPU?): 2.2.2+cu121 (True)
- Tensorflow version (GPU?): not installed (NA)
- Flax version (CPU?/GPU?/TPU?): not installed (NA)
- Jax version: not installed
- JaxLib version: not installed
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



In [8]:
import os
import torch
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [26]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.2'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [27]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "bfloat16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Device partition
device_map = "auto"

In [28]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [29]:
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage = True,
    device_map = device_map
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [163]:
#################################################################
# Base inference
#################################################################

prompt = """Explícame el artículo 5. Régimen de libre competencia"""

# CUDA: Para programar directamente la GPU
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

Explícame el artículo 5. Régimen de libre competencia.

El artículo 5 de la Constitución Española de 1978 establece el régimen de libre competencia en España. Según este artículo, todos los españoles tienen derecho a ejercer libremente cualquier actividad económica, salvo aquellas que sean expresamente reservadas a los estados o a las comunidades autónomas por la Constitución o por las leyes.

Además, el artículo 5 garantiza la no discriminación entre españoles en materia económica, lo que significa que todos tienen la misma oportunidad de participar en el mercado laboral y empresarial, sin distinción de origen, sexo, edad, religión o cualquier otra característica personal.

Por otro lado, el artículo 5 también establece la libertad de precios, lo que significa que los precios de los bienes y servicios se determinan libremente en el mercado, sin intervención directa del Estado. Sin embargo, el Estado puede estable


## LangChain

In [128]:
import pypdf
from langchain.document_loaders import DirectoryLoader, PyPDFLoader, PDFMinerLoader
from pypdf import PdfReader

CHROMA_PATH = "chroma4"
DATA_PATH = "data"

def get_pdf_text():
    loader = PyPDFLoader("data/BOE-A-2022-10757.pdf")
    pages = loader.load_and_split()
    
    return pages

In [129]:
documents = get_pdf_text()
print(documents[0])

page_content='I. DISPOSICIONES GENERALES\nJEFATURA DEL ESTADO\n10757 Ley 11/2022, de 28 de junio, General de Telecomunicaciones.\nFELIPE VI\nREY DE ESPAÑA\nA todos los que la presente vieren y entendieren.\nSabed: Que las Cortes Generales han aprobado y Yo vengo en sancionar la \nsiguiente ley:\nPREÁMBULO\nI\nTítulo I.\u2003Disposiciones generales.\nArtículo 1.\u2003Objeto y ámbito de aplicación de la ley.\nArtículo 2.\u2003Las telecomunicaciones como servicios de interés general.\nArtículo 3.\u2003Objetivos y principios de la ley.\nArtículo 4. \u2003Servicios de telecomunicaciones para la seguridad nacional, la defensa \nnacional, la seguridad pública, la seguridad vial y la protección civil.\nTítulo II. \u2003Suministro de redes y prestación de servicios de comunicaciones \nelectrónicas en régimen de libre competencia.\nCapítulo I.\u2003Disposiciones generales.\nArtículo 5.\u2003Régimen de libre competencia.\nArtículo 6.\u2003 Requisitos exigibles para el suministro de las redes y la

In [130]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0,
        length_function=len,
        add_start_index=True
    )
    
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    # Cogemos un chunk aleatorio
    document = chunks[10]
    print(document.page_content) # Printeamos su contenido
    print(document.metadata) # Y su metadata (Fichero al que pertenece y donde empieza 'start_index')
    
    return chunks

In [131]:
chunks = split_text(documents)

Split 247 documents into 1492 chunks.
Capítulo II.  Derechos de los operadores y despliegue de redes públicas de 
comunicaciones electrónicas.
Sección 1.ª  Derechos de los operadores a la ocupación del dominio público, a ser 
beneficiarios en el procedimiento de expropiación forzosa y al establecimiento a su favor 
de servidumbres y de limitaciones a la propiedad.
Artículo 44. Derecho de ocupación de la propiedad privada.
Artículo 45. Derecho de ocupación del dominio público.
{'source': 'data/BOE-A-2022-10757.pdf', 'page': 2, 'start_index': 0}


In [132]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
import shutil
import chromadb

def save_to_chroma(chunks):
    
    # Clear out the database first if already exists.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
    
    #embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    embeddings = SentenceTransformerEmbeddings(model_name="distiluse-base-multilingual-cased-v2")
    
    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, 
        embeddings, 
        persist_directory=CHROMA_PATH
    )
    
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

    return embeddings

In [133]:
embeddings = save_to_chroma(chunks)

Saved 1492 chunks to chroma4.


In [134]:
#Function to turn the word apple to a vector
from langchain.evaluation import load_evaluator

vector = embeddings.embed_query("almendras")
#print(f"Vector for 'apple': {vector}")
print(f"Vector length: {len(vector)}")

# Compare vector of two words (distance between two words)
# Esto nos ayudará a saber si dos palabras estas correlacionadas
# Si comparamos dos palabras iguales, la distancia será prácticamente 0
# Utilza por defecto OpenAI embeddings!
evaluator = load_evaluator("embedding_distance", embeddings=embeddings)

words = ("apple", "iphone")
x = evaluator.evaluate_strings(prediction=words[0], reference=words[1])
print(f"Comparing ({words[0]}, {words[1]}): {x}")

Vector length: 512
Comparing (apple, iphone): {'score': 0.30508241313171336}


In [135]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

def load_model():
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.1,
        repetition_penalty=1.1,
        return_full_text=True,
        max_new_tokens=300,
        do_sample=True
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf

In [136]:
llm = load_model()

In [137]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain

def query_data2(embeddings, llm):
    
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
    
    retriever = db.as_retriever()

    prompt_template = """
    ### [INST] Instrucción: Eres un experto en jurisdiccion, responde a la pregunta según tus conocimientos de jurisdiccion y el siguiente contexto:
    
    {context}
    
    ### PREGUNTA:
    {question} (responde en castellano) [/INST]
     """
    
    promptTemplate = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template
    )
    
    llm_chain = LLMChain(llm=llm, prompt=promptTemplate)

    # RunnablePassthrough para pasar la query al siguiente step en la chain
    rag_chain = ( 
     {"context": retriever, "question": RunnablePassthrough()}
        | llm_chain
    )

    return rag_chain

In [138]:
#db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
#query = """En que consiste el Artículo 13"""
#results = db.similarity_search_with_relevance_scores(query, k=3)
#context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
#print(context_text)

In [159]:
rag_chain = query_data2(embeddings, llm)
query = """Explícame el artículo 5. Régimen de libre competencia"""
answer = rag_chain.invoke(query)

In [160]:
answer['context']

[Document(page_content='normativa específica sobre las telecomunicaciones relacionadas con el orden público, la \nseguridad pública, la defensa nacional y la seguridad nacional.\nTÍTULO II\nSuministro de redes y prestación de servicios de comunicaciones electrónicas \nen régimen de libre competencia\nCAPÍTULO I\nDisposiciones generales\nArtículo 5. \u2003Régimen de libre competencia.\nLa instalación y explotación de las redes y la prestación de los servicios de', metadata={'page': 16, 'source': 'data/BOE-A-2022-10757.pdf', 'start_index': 866}),
 Document(page_content='Artículo 4. \u2003Servicios de telecomunicaciones para la seguridad nacional, la defensa \nnacional, la seguridad pública, la seguridad vial y la protección civil.\nTítulo II. \u2003Suministro de redes y prestación de servicios de comunicaciones \nelectrónicas en régimen de libre competencia.\nCapítulo I.\u2003Disposiciones generales.\nArtículo 5.\u2003Régimen de libre competencia.\nArtículo 6.\u2003 Requisitos exigibles 

In [161]:
answer['text']

'\n    ### [INST] Instrucción: Eres un experto en jurisdiccion, responde a la pregunta según tus conocimientos de jurisdiccion y el siguiente contexto:\n    \n    [Document(page_content=\'normativa específica sobre las telecomunicaciones relacionadas con el orden público, la \\nseguridad pública, la defensa nacional y la seguridad nacional.\\nTÍTULO II\\nSuministro de redes y prestación de servicios de comunicaciones electrónicas \\nen régimen de libre competencia\\nCAPÍTULO I\\nDisposiciones generales\\nArtículo 5. \\u2003Régimen de libre competencia.\\nLa instalación y explotación de las redes y la prestación de los servicios de\', metadata={\'page\': 16, \'source\': \'data/BOE-A-2022-10757.pdf\', \'start_index\': 866}), Document(page_content=\'Artículo 4. \\u2003Servicios de telecomunicaciones para la seguridad nacional, la defensa \\nnacional, la seguridad pública, la seguridad vial y la protección civil.\\nTítulo II. \\u2003Suministro de redes y prestación de servicios de comunica

In [162]:
indent = 0
for key, value in answer.items():
    if isinstance(value, dict):
        print('  ' * indent + f'{key}:')
        pretty_print_dict(value, indent + 1)
    else:
        print('  ' * indent + f'{key}: {value}')

context: [Document(page_content='normativa específica sobre las telecomunicaciones relacionadas con el orden público, la \nseguridad pública, la defensa nacional y la seguridad nacional.\nTÍTULO II\nSuministro de redes y prestación de servicios de comunicaciones electrónicas \nen régimen de libre competencia\nCAPÍTULO I\nDisposiciones generales\nArtículo 5. \u2003Régimen de libre competencia.\nLa instalación y explotación de las redes y la prestación de los servicios de', metadata={'page': 16, 'source': 'data/BOE-A-2022-10757.pdf', 'start_index': 866}), Document(page_content='Artículo 4. \u2003Servicios de telecomunicaciones para la seguridad nacional, la defensa \nnacional, la seguridad pública, la seguridad vial y la protección civil.\nTítulo II. \u2003Suministro de redes y prestación de servicios de comunicaciones \nelectrónicas en régimen de libre competencia.\nCapítulo I.\u2003Disposiciones generales.\nArtículo 5.\u2003Régimen de libre competencia.\nArtículo 6.\u2003 Requisitos ex